### This only works on cloud computer

In [ ]:
### For stock trading part
from src.function import *
import argparse

parser = argparse.ArgumentParser()

parser.add_argument('-i', metavar='N', 
                    type=int, dest='a_index',
                    help='an integer for the train index selection')

parser.add_argument('-m', action='store',
                    dest='a_method',
                    help='Store a classification method,\
                    rfc: random forest,\
                    knn: K nearest neighbor,\
                    svm: Support vector machine,\
                    lg: Logistic regression,\
                    xgb: Xgboost,\
                    esm: Ensemble,\
                    mlp: Mutiple Layer Propagation,\
                    rnn: Recurrent Neural Network,\
                    lstm: Long Short Term Memory\
                    ')

parser.add_argument('-k', metavar='K', 
                    type=int, dest='a_pair',
                    help='an integer for the number of stocks in each pair')

parse_args = parser.parse_args()
base_os = os.getcwd()+'/clean_data'

#train_ind = 2
train_ind = parse_args.a_index
sel_method = parse_args.a_method
sel_k = parse_args.a_pair

train_file = "/set"+str(train_ind)+"_Feature.csv"
target_file = "/set"+str(train_ind)+"_Target.csv"
X = pd.read_csv(base_os+train_file,index_col=0)
y = pd.read_csv(base_os+target_file, index_col=0)


prefix_market = 'sp500'
#prefix_market = 'FTSE100'
base_os = os.getcwd()+'/data'
#os1 = os.path.join(base_os, prefix_market,'cleaned_data')
#os2 = os.path.join(base_os, prefix_market,'download_data_revise')
start_date = '2005-01-01'
end_date = '2020-01-01'
##load all data
filename = prefix_market + "_matrix_utd.csv"
origdat_matrix = pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='symbol')
origdat_matrix.columns = origdat_matrix.columns.astype('datetime64[ns]')
print("The size of \"{}\" is: {}".format(filename, origdat_matrix.shape))

filename = prefix_market + "_all_download_data_01-01-2005_12-31-2019_Adj Close.csv"
origdat = pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='date',parse_dates = True)
##clean origdat
origdat = rm_extreme_price(origdat)
print("The size of \"{}\" is: {}".format(filename, origdat.shape))

filename = prefix_market + "_all_download_data_01-01-2005_12-31-2019_Adj Open.csv"
origdat_o = pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='date',parse_dates = True)
print("The size of \"{}\" is: {}".format(filename, origdat_o.shape))

filename = prefix_market + "_index_data_01-01-2000_05-20-2020.csv"
sp500 =  pd.read_csv(os.path.join(base_os,filename),sep=",",index_col='date',parse_dates = True)
print("The size of \"{}\" is: {}".format(filename, sp500.shape))

sp500 = pd.DataFrame(sp500['Close'])
sp500.columns = ['SP500']
#sp500.head()
origdat = origdat.merge(sp500, how = 'left', left_index=True, right_index=True)



#set for training periods
first_period1 = [datetime.strptime(item,'%Y-%m-%d') for item in ['2005-01-01', '2008-01-01', '2009-01-01']]
nrows1 = diff_year(datetime.strptime('2009-01-01','%Y-%m-%d'),datetime.strptime('2020-01-01','%Y-%m-%d'))+1
periods1=[]
for i in range(nrows1):
    periods1.append(list(map(lambda period: add_years(period,i), first_period1)))
    
periods1 = pd.DataFrame(periods1)
periods1.columns = ['train_start','trad_start','trad_end']
periods1 = periods1.apply(pd.to_datetime)

#set_dict1
set_dict1 = data_quality(nrows1,periods1,origdat_matrix,origdat) 
m = list(range(1,21))
for k in range(2,13):
    m.append(k*20)


cur_key = set_dict1[periods1.iloc[train_ind,0]]
start_date = periods1.iloc[train_ind,0]
end_date = periods1.iloc[train_ind,1]

#print(cur_key[-10:]+["SP500"])
cur_dat = origdat.loc[start_date:end_date,cur_key]
#cur_dat.fillna(method = 'ffill',inplace=True)
cur_dat.fillna(method = 'ffill',inplace=True)

print("Start Trading!")
trad_key = set_dict1[periods1.iloc[train_ind,0]]
start_date = periods1.iloc[train_ind,0]
end_date = periods1.iloc[train_ind,2]

#print(cur_key[-10:]+["SP500"])
total_dat = origdat.loc[start_date:end_date,cur_key]

#cur_dat.fillna(method = 'ffill',inplace=True)
total_dat.fillna(method = 'ffill',inplace=True)
#use mean to replace 0
total_dat = total_dat.apply(lambda x: x.replace(0,x.mean()), axis = 0)
total_dat.head()

start_dat1 = cur_dat.shape[0]
end_dat1 = total_dat.shape[0]

avg_ret =[]

if __name__ == "__main__":
    file_name = os.getcwd()+'/result_trade/'+sel_method+"/train_set_"+str(train_ind)+"_trading_sel"+str(sel_k)+".txt"
    with open(file_name, 'w') as f:
        print("Running train index", train_ind, file = f)
        print("Use method", sel_method, file= f)
        
    avg_ret, avg_pred_ac, avg_cnf = train_trade_once(dat=origdat,set_dict=set_dict1,train_ind=train_ind, periods=periods1,X=X, y=np.ravel(y), m=m, method=sel_method, cv="False",sel_k=sel_k)
    with open(file_name, 'w') as f:
        print("Average daily return is", avg_ret, file = f)
        print("Mean of Average daily return is", np.mean(avg_ret))
    
    f2 = plt.figure(figsize=(10,7))
    plt.hist(avg_ret,histtype='stepfilled',alpha=0.5)
    plt.axvline(np.mean(avg_ret),color='r')
    plt.text(np.mean(avg_ret),50,'mean='+ str(round(np.mean(avg_ret),4)),rotation=0)
    plt.show()
    plt_name = os.getcwd()+'/result_trade/'+sel_method+"/train_set_"+str(train_ind)+"_trading_sel"+str(sel_k)+".pdf"
    f2.savefig(plt_name, bbox_inches='tight')